In [65]:
import pandas as pd
import numpy as np

basic = pd.read_csv("data/metadata.csv", index_col=0)
basic.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"('Animation', 'Comedy', 'Family')",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"('English',)",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"('Adventure', 'Family', 'Fantasy')",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"('English', 'Français')",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,Grumpy Old Men Collection,0,"('Comedy', 'Romance')",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"('English',)",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"('Comedy', 'Drama', 'Romance')",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"('English',)",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,Father of the Bride Collection,0,"('Comedy',)",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"('English',)",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [66]:
len(basic['title'].unique())

42277

In [106]:
o = basic[basic['original_language'] == 'en'][['id', 'title', 'overview']].reset_index(drop=True)

In [107]:
o.loc[:, 'overview'] = o['overview'].fillna(value='')
o

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
30058,390959,Shadow of the Blair Witch,"In this true-crime documentary, we delve into ..."
30059,289923,The Burkittsville 7,A film archivist revisits the story of Rustin ...
30060,222848,Caged Heat 3000,It's the year 3000 AD. The world's most danger...
30061,227506,Satan Triumphant,"In a small town live two brothers, one a minis..."


In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', strip_accents='unicode')
tfidf_matrix = tfidf.fit_transform(o['overview'])

tfidf_matrix.shape

(30063, 55968)

In [109]:
def get_index(id):
    return o[o['id'] == id].index[0]

In [110]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similarities(id):
    return cosine_similarity(tfidf_matrix, tfidf_matrix[get_index(id)]).reshape(-1)

In [111]:
def get_id(index):
    return o.loc[index]['id']

In [103]:
def get_similar(id):
    return np.argsort(get_similarities(id))[-50:][::-1]

In [104]:
def util_get_id_from_title(title):
    return o[o['title'] == 'The Dark Knight Rises']

,index,id,title,overview
13653,18252,49026,The Dark Knight Rises,Following the death of District Attorney Harve...


In [105]:
o.iloc[get_similar(49026)]

,index,id,title,overview
13653,18252,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
9915,12481,155,The Dark Knight,Batman raises the stakes in his war on crime. ...
132,150,414,Batman Forever,The Dark Knight of Gotham City confronts a das...
1163,1328,364,Batman Returns,"Having defeated the Joker, Batman now faces th..."
11947,15511,40662,Batman: Under the Red Hood,Batman faces his ultimate challenge as the mys...
541,585,268,Batman,The Dark Knight of Gotham City begins his war ...
15618,21192,29751,Batman Unmasked: The Psychology of the Dark Kn...,Delve into the world of Batman and the vigilan...
7670,9230,16234,Batman Beyond: Return of the Joker,"The Joker is back with a vengeance, and Gotham..."
13514,18035,69735,Batman: Year One,Two men come to Gotham City: Bruce Wayne after...
14677,19790,123025,"Batman: The Dark Knight Returns, Part 1",Batman has not been seen for ten years. A new ...
